In [1]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse
from sys import platform
%load_ext autoreload
%autoreload 2

In [2]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())


print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA TITAN V


In [4]:
# iv: image option
length = 440
channel = 128
min_CNN = 200
n_classes = 40
classes = range(n_classes)


In [6]:
if platform == "linux" or platform == "linux2":
    torch_models_dir = r"/media/titan/AI Research1/Data/CVPR2017"
elif platform == "win32":
    torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
block_splits_all, block_splits_single, eeg_14_70, eeg_55_95, eeg_5_95, eeg_raw = os.listdir(torch_models_dir)
print(os.listdir(torch_models_dir))

['block_splits_by_image_all.pth', 'block_splits_by_image_single.pth', 'eeg_14_70_std.pth', 'eeg_55_95_std.pth', 'eeg_5_95_std.pth', 'eeg_signals_raw_with_mean_std.pth']


In [7]:
eeg_dataset = os.path.join(torch_models_dir, eeg_5_95)
splits_all_path = os.path.join(torch_models_dir, block_splits_all)
splits_single_path = os.path.join(torch_models_dir, block_splits_single)
# splits_path = os.path.join(torch_models_dir, splits_shuffled_path)
print(eeg_dataset,'\n', splits_all_path, '\n', splits_single_path)

/media/titan/AI Research1/Data/CVPR2017/eeg_5_95_std.pth 
 /media/titan/AI Research1/Data/CVPR2017/block_splits_by_image_all.pth 
 /media/titan/AI Research1/Data/CVPR2017/block_splits_by_image_single.pth


In [8]:
splits_all = torch.load(splits_all_path)
splits_single = torch.load(splits_single_path)

In [9]:
print(len(splits_all['splits']))
print(len(splits_all['splits'][0]))

print(len(splits_all['splits'][5]['train']))
print(len(splits_all['splits'][5]['val']))
print(len(splits_all['splits'][5]['test']))
print(splits_all['splits'][0]['train'][:40])
print(splits_all['splits'][1]['train'][:40])
print(splits_all['splits'][2]['train'][:10])
print(splits_all['splits'][3]['train'][:10])


6
3
7984
1996
1985
[0, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 27, 29, 33, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 48, 51, 52, 54, 55]
[1, 2, 3, 4, 6, 8, 9, 12, 13, 20, 25, 26, 27, 28, 30, 32, 33, 35, 37, 38, 39, 40, 44, 45, 46, 50, 52, 54, 56, 58, 59, 60, 62, 65, 68, 72, 73, 74, 76, 81]
[2, 3, 4, 5, 6, 7, 8, 10, 11, 13]
[1, 2, 4, 7, 9, 10, 12, 13, 14, 15]


In [17]:
# print(splits_single)
# print(splits_single['splits'])
print(len(splits_single['splits'][0]['train']))
print(len(splits_single['splits'][0]['val']))
print(len(splits_single['splits'][0]['test']))

669
167
164


In [15]:
eeg_loaded = torch.load(eeg_dataset)

In [18]:
print(len(eeg_loaded))
print(eeg_loaded.keys())
dataset, labels, images = [eeg_loaded[k] for k in eeg_loaded.keys()]
print(len(labels))
print(len(images))
print(len(dataset))

print(*labels)
print(images[0])
print(dataset[0]['eeg'].shape)
print(dataset[0])

3
dict_keys(['dataset', 'labels', 'images'])
40
1996
11965
n02389026 n03888257 n03584829 n02607072 n03297495 n03063599 n03792782 n04086273 n02510455 n11939491 n02951358 n02281787 n02106662 n04120489 n03590841 n02992529 n03445777 n03180011 n02906734 n07873807 n03773504 n02492035 n03982430 n03709823 n03100240 n03376595 n03877472 n03775071 n03272010 n04069434 n03452741 n03792972 n07753592 n13054560 n03197337 n02504458 n02690373 n03272562 n04044716 n02124075
n02951358_31190
torch.Size([128, 500])
{'eeg': tensor([[-0.0098,  0.0195,  0.0620,  ...,  0.0638,  0.0120, -0.0118],
        [-0.0045,  0.1303,  0.2673,  ...,  0.0894,  0.0342, -0.0082],
        [ 0.0215, -0.2017, -0.4305,  ..., -0.2022, -0.0940,  0.0188],
        ...,
        [ 0.0160,  0.0707,  0.1005,  ...,  0.2066,  0.1156,  0.0036],
        [-0.0046, -0.0084, -0.0119,  ...,  0.0007, -0.0026, -0.0053],
        [ 0.0040,  0.0419,  0.0665,  ...,  0.0765,  0.0309, -0.0063]]), 'image': 0, 'label': 10, 'subject': 4}


In [12]:
opt = {
    # Dataset options
#     "iv": "image",
#     "offset": None,
    "results_file": "results.pkl",
    "subject": 0,
    "time_low": 20,
    "time_high": 460,
#     "run": "none",
    "eeg_dataset": eeg_dataset,
    "model_type": "model10",
    "splits_path": splits_all_path,
    "split_num": 0,
    "split_name": "train",
#     "fold": 5,
    #Training options
    "batch_size": 16,
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"from-scratch",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt)

Namespace(results_file='results.pkl', subject=0, time_low=20, time_high=460, eeg_dataset='/media/titan/AI Research1/Data/CVPR2017/eeg_5_95_std.pth', model_type='model10', splits_path='/media/titan/AI Research1/Data/CVPR2017/block_splits_by_image_all.pth', split_num=0, split_name='train', batch_size=16, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier=None)


In [13]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
# from data_loader import EEGDataset, Splitter, SplitterWithData
from data_loader_CVPR2017 import EEGDataset, Splitter
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_generator import Classifier
from EEG_Encoder.net_trainer import net_trainer
from p_values import *
from torchinfo import summary

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
def load_dataset(
#              offset,
             eeg_dataset,
             splits_path,
             split_num, # (0-5) - 6 fold cross validation
#              total, 
#              classes,
#              classifier,
             batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             kind="from-scratch"):        
    # Load dataset
    dataset = EEGDataset(opt, eeg_dataset)
    print("DONE: LOAD DATASET")
#     # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
#     if kind=="from-scratch":
#         relabel = False
#     if kind=="incremental":
#         relabel = False
#     if kind=="no-model-file":
#         relabel = True
    splitter = {split: Splitter(dataset,
                    splits_path,
                    split_num,
                    split_name=split) for split in ["train", "val", "test"]}
    loaders = {split: DataLoader(
                        splitter[split],
                        batch_size = batch_size,
                        drop_last = False,
                        shuffle = True)
                    for split in ["train", "val", "test"]}
    channel_idx = None    
    print("DONE: Create loaders for model")            
    return dataset, loaders, splitter

In [15]:
# Options
opt.classifier = "LSTM4"
opt.batch_size = 16
# opt.kind = "from-scratch"
# opt.run = "imagenet40-1000"
# opt.fold = 5

In [16]:
dataset, loaders, splitter = load_dataset(
#              offset,
             opt.eeg_dataset,
             opt.splits_path,
             opt.split_num, # (0-5) - 6 fold cross validation
#              total, 
#              classes,
#              classifier,
             opt.batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [17]:
# loaders: divide the splits data in each fold with batch_size
# Each fold has {train: 8000 idx, val: 2000 idx, test: 2000 idx}
# Each loader batch has {train: 2000 idx, val: 250 idx, test: 250 idx}
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
# for i, (input, target) in enumerate(loaders["train"]):
#     if i<20:
#         print(f"{i+1}: Target size: {target.size()}; input size: {input.size()}")
for i in range(0, 40):
    eeg, label_val = splitter["val"][i]
    eeg, label_train = splitter["train"][i]
    print(f"{i+1}: Label val: {label_val}; label train: {label_train}")


<class 'data_loader_CVPR2017.EEGDataset'>
<class 'dict'>
3 [498, 125, 125]
1: Label val: 37; label train: 10
2: Label val: 38; label train: 10
3: Label val: 11; label train: 30
4: Label val: 10; label train: 25
5: Label val: 7; label train: 18
6: Label val: 35; label train: 3
7: Label val: 13; label train: 8
8: Label val: 2; label train: 11
9: Label val: 14; label train: 18
10: Label val: 2; label train: 28
11: Label val: 33; label train: 38
12: Label val: 26; label train: 20
13: Label val: 34; label train: 3
14: Label val: 18; label train: 28
15: Label val: 11; label train: 23
16: Label val: 32; label train: 0
17: Label val: 6; label train: 34
18: Label val: 22; label train: 20
19: Label val: 24; label train: 23
20: Label val: 24; label train: 39
21: Label val: 29; label train: 0
22: Label val: 3; label train: 34
23: Label val: 25; label train: 21
24: Label val: 11; label train: 39
25: Label val: 22; label train: 6
26: Label val: 36; label train: 26
27: Label val: 30; label train: 20


In [18]:
net, nonclasses = Classifier(
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt.kind)
# print(len(nonclasses))
summary(net, input_size=(1,128, 440))

DONE: CREATE TORCH CLASSIFIER
classifier_LSTM(
  (lstm): LSTM(128, 128, batch_first=True)
  (output1): Linear(in_features=128, out_features=128, bias=True)
  (relu): ReLU()
  (output2): Linear(in_features=128, out_features=40, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
classifier_LSTM                          [1, 40]                   --
├─LSTM: 1-1                              [1, 440, 128]             132,096
├─Linear: 1-2                            [1, 128]                  16,512
├─ReLU: 1-3                              [1, 128]                  --
├─Linear: 1-4                            [1, 40]                   5,160
Total params: 153,768
Trainable params: 153,768
Non-trainable params: 0
Total mult-adds (M): 58.14
Input size (MB): 0.23
Forward/backward pass size (MB): 0.45
Params size (MB): 0.62
Estimated Total Size (MB): 1.29

In [19]:
model_path = (   "CVPR2017-" +
                  opt.classifier+
                  "-"+
                  str(length)+
                  "-"+
                  str(channel) )
print(model_path)

CVPR2017-LSTM4-440-128


In [20]:
print(opt)

Namespace(results_file='results.pkl', subject=0, time_low=20, time_high=460, eeg_dataset='/media/titan/AI Research1/Data/CVPR2017/eeg_5_95_std.pth', model_type='model10', splits_path='/media/titan/AI Research1/Data/CVPR2017/block_splits_by_image_all.pth', split_num=0, split_name='train', batch_size=16, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier='LSTM4')


In [21]:
channel_idx=None
non_classes=None
if opt.kind=="from-scratch":
    results = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path,
            print_every_train = 100,
            print_every_val = 125)

Epoch 1
Train Batch 100 (every 100 batch): Loss=3.6923; accuracy=0.0306
Train Batch 200 (every 100 batch): Loss=3.6666; accuracy=0.0309
Train Batch 300 (every 100 batch): Loss=3.6078; accuracy=0.0317
Train Batch 400 (every 100 batch): Loss=3.5138; accuracy=0.0337
Epoch 1 summary: train_loss: 3.6511 | train_acc: 0.0353 | val_loss: 3.5452 | val_acc: 0.0465
Epoch 2
Train Batch 100 (every 100 batch): Loss=3.4069; accuracy=0.0600
Train Batch 200 (every 100 batch): Loss=3.2191; accuracy=0.0569
Train Batch 300 (every 100 batch): Loss=3.0580; accuracy=0.0596
Train Batch 400 (every 100 batch): Loss=3.1633; accuracy=0.0630
Epoch 2 summary: train_loss: 3.3710 | train_acc: 0.0634 | val_loss: 3.3436 | val_acc: 0.0645
Epoch 3
Train Batch 100 (every 100 batch): Loss=3.1628; accuracy=0.0981
Train Batch 200 (every 100 batch): Loss=3.3368; accuracy=0.0969
Train Batch 300 (every 100 batch): Loss=3.0836; accuracy=0.0983
Train Batch 400 (every 100 batch): Loss=3.1526; accuracy=0.0978
Epoch 3 summary: train

Train Batch 100 (every 100 batch): Loss=1.1166; accuracy=0.8269
Train Batch 200 (every 100 batch): Loss=0.4700; accuracy=0.8137
Train Batch 300 (every 100 batch): Loss=0.9358; accuracy=0.7950
Train Batch 400 (every 100 batch): Loss=0.3829; accuracy=0.7736
Epoch 24 summary: train_loss: 0.7468 | train_acc: 0.7617 | val_loss: 4.0970 | val_acc: 0.1593
Epoch 25
Train Batch 100 (every 100 batch): Loss=0.6941; accuracy=0.8050
Train Batch 200 (every 100 batch): Loss=0.6288; accuracy=0.7966
Train Batch 300 (every 100 batch): Loss=0.8093; accuracy=0.7927
Train Batch 400 (every 100 batch): Loss=0.6758; accuracy=0.7833
Epoch 25 summary: train_loss: 0.6764 | train_acc: 0.7843 | val_loss: 4.2614 | val_acc: 0.1551
Epoch 26
Train Batch 100 (every 100 batch): Loss=0.4501; accuracy=0.8338
Train Batch 200 (every 100 batch): Loss=0.3928; accuracy=0.8309
Train Batch 300 (every 100 batch): Loss=0.4507; accuracy=0.8269
Train Batch 400 (every 100 batch): Loss=0.7767; accuracy=0.8245
Epoch 26 summary: train_lo

Train Batch 100 (every 100 batch): Loss=0.4243; accuracy=0.9425
Train Batch 200 (every 100 batch): Loss=0.3916; accuracy=0.9209
Train Batch 300 (every 100 batch): Loss=0.1319; accuracy=0.9219
Train Batch 400 (every 100 batch): Loss=0.0547; accuracy=0.9233
Epoch 47 summary: train_loss: 0.2552 | train_acc: 0.9248 | val_loss: 6.7415 | val_acc: 0.1426
Epoch 48
Train Batch 100 (every 100 batch): Loss=0.0600; accuracy=0.9262
Train Batch 200 (every 100 batch): Loss=0.0581; accuracy=0.9216
Train Batch 300 (every 100 batch): Loss=0.0775; accuracy=0.9198
Train Batch 400 (every 100 batch): Loss=0.2262; accuracy=0.9184
Epoch 48 summary: train_loss: 0.2879 | train_acc: 0.9166 | val_loss: 6.7179 | val_acc: 0.1455
Epoch 49
Train Batch 100 (every 100 batch): Loss=0.0315; accuracy=0.9594
Train Batch 200 (every 100 batch): Loss=0.1388; accuracy=0.9559
Train Batch 300 (every 100 batch): Loss=0.0237; accuracy=0.9550
Train Batch 400 (every 100 batch): Loss=0.1262; accuracy=0.9539
Epoch 49 summary: train_lo

Train Batch 100 (every 100 batch): Loss=0.1781; accuracy=0.9106
Train Batch 200 (every 100 batch): Loss=0.1746; accuracy=0.9087
Train Batch 300 (every 100 batch): Loss=0.4359; accuracy=0.9023
Train Batch 400 (every 100 batch): Loss=0.0965; accuracy=0.9069
Epoch 70 summary: train_loss: 0.2898 | train_acc: 0.9144 | val_loss: 7.9914 | val_acc: 0.1365
Epoch 71
Train Batch 100 (every 100 batch): Loss=0.0938; accuracy=0.9663
Train Batch 200 (every 100 batch): Loss=0.0850; accuracy=0.9659
Train Batch 300 (every 100 batch): Loss=0.0175; accuracy=0.9650
Train Batch 400 (every 100 batch): Loss=0.0233; accuracy=0.9672
Epoch 71 summary: train_loss: 0.1064 | train_acc: 0.9683 | val_loss: 7.9082 | val_acc: 0.1415
Epoch 72
Train Batch 100 (every 100 batch): Loss=0.0267; accuracy=0.9794
Train Batch 200 (every 100 batch): Loss=0.0236; accuracy=0.9856
Train Batch 300 (every 100 batch): Loss=0.0082; accuracy=0.9871
Train Batch 400 (every 100 batch): Loss=0.0145; accuracy=0.9869
Epoch 72 summary: train_lo

Train Batch 100 (every 100 batch): Loss=0.0119; accuracy=0.9969
Train Batch 200 (every 100 batch): Loss=0.0100; accuracy=0.9947
Train Batch 300 (every 100 batch): Loss=0.0152; accuracy=0.9942
Train Batch 400 (every 100 batch): Loss=0.0074; accuracy=0.9917
Epoch 93 summary: train_loss: 0.0359 | train_acc: 0.9915 | val_loss: 8.5207 | val_acc: 0.1585
Epoch 94
Train Batch 100 (every 100 batch): Loss=0.0168; accuracy=0.9944
Train Batch 200 (every 100 batch): Loss=0.0055; accuracy=0.9953
Train Batch 300 (every 100 batch): Loss=0.0571; accuracy=0.9956
Train Batch 400 (every 100 batch): Loss=0.0071; accuracy=0.9958
Epoch 94 summary: train_loss: 0.0205 | train_acc: 0.9961 | val_loss: 8.4995 | val_acc: 0.1623
Epoch 95
Train Batch 100 (every 100 batch): Loss=0.0086; accuracy=0.9994
Train Batch 200 (every 100 batch): Loss=0.0034; accuracy=0.9950
Train Batch 300 (every 100 batch): Loss=0.0163; accuracy=0.9919
Train Batch 400 (every 100 batch): Loss=0.1134; accuracy=0.9906
Epoch 95 summary: train_lo

In [27]:
# val = results["val_acc"]
# # test = results["test_acc"]

# print("Validation accuracy: ", val)
# # print("Test accuracy: ", test)

In [ ]:
print(accuracy_val)